In [ ]:
'''
Run on virtual environment: Snow_env with the following packages installed
***tested on Python 3.9.12, does not work with python 3.10***

Load all dependencies via cmd and pay special attention to version requirements.

Steps for installing virtual env: https://janakiev.com/blog/jupyter-virtual-envs/

1st geopandas: https://towardsdatascience.com/geopandas-installation-the-easy-way-for-windows-31a666b3610f
Here are binaries: https://www.lfd.uci.edu/~gohlke/pythonlibs/
2nd: rioxarray
3rd: rasterio
tables
sklearn
earthpy
numpy 1.22
pyshp 2.1
h5py
tqdm
matplotlib 3.5
seaborn
tensorflow



joblib
pillow
panda
pyproj
shapely
kiwisolver
fonttools
pillow
mpl_toolkits - pip install --user basemap
**git clone https://github.com/matplotlib/matplotlib
cd matplotlib
pip install -e .**
tables
pytables
netCDF4 **https://unidata.github.io/netcdf4-python/

Data Assimilation of snotel data
https://snowex-hackweek.github.io/website/tutorials/geospatial/SNOTEL_query.html

'''

import os
import National_Snow_Model
import NSM_SCA
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#set path directory
os.getcwd()
os.chdir('..')  # TODO find a way around this, it causes issues when rerunning the code
cwd = os.getcwd()
cwd

## Running the NSM to create retrospective datasets

The following datasets need to be 'created' within the Predictions folder, replacing YEAR with beginning of the desired water year:
* predictionsYEAR-09-24.h5
* submission_format_YEAR-09-24.csv

The following datasets need to be 'created' within the Pre_Processed_DA folder, replacing YEAR with beginning of the desired water year:
* ground_measures_features_YEAR-09-24.csv

### Rename columns in h5 file

In [ ]:
from datetime import date, timedelta

#the year of the original file
prev_year = '2020'
prev_date = prev_year + '-09-24'

#input the new year of choice
water_year = '2021'
new_date = water_year + '-09-24'

# TODO do the CSV files need to be updated as well?

#for h5 files
Region_list = ['N_Sierras',
                       'S_Sierras_High',
                       'S_Sierras_Low',
                       'Greater_Yellowstone',
                       'N_Co_Rockies',
                       'SW_Mont',
                       'SW_Co_Rockies',
                       'GBasin',
                       'N_Wasatch',
                       'N_Cascade',
                       'S_Wasatch',
                       'SW_Mtns',
                       'E_WA_N_Id_W_Mont',
                       'S_Wyoming',
                       'SE_Co_Rockies',
                       'Sawtooth',
                       'Ca_Coast',
                       'E_Or',
                       'N_Yellowstone',
                       'S_Cascade',
                       'Wa_Coast',
                       'Greater_Glacier',
                       'Or_Coast'
                      ]
SWE_new = {}
for region in Region_list:
    #The below file will serve as a starting point
    SWE_new[region] = pd.read_hdf(cwd+'/Predictions/predictions'+prev_year+'-09-24.h5', key = region)
    SWE_new[region].rename(columns = {prev_date:new_date}, inplace = True)
    SWE_new[region].to_hdf(cwd+'/Predictions/predictions'+water_year+'-09-24.h5', key = region)

In [ ]:
#define start and end date for list of dates
start_dt = date(2020, 10, 1)
end_dt = date(2021, 7, 31)

In [ ]:
#can be altered to create list every n number of days by changing 7 to desired skip length
def daterange(start_date, end_date):
     for n in range(0, int((end_date - start_date).days) + 1, 7):
        yield start_date + timedelta(n)
         
#create empty list to store dates
datelist = []

#append dates to list
for dt in daterange(start_dt, end_dt):
    #print(dt.strftime("%Y-%m-%d"))
    dt=dt.strftime('%Y-%m-%d')
    datelist.append(dt)

In [ ]:
datelist

## Run model

In [ ]:
#run the model through all time (data acqusition already completed)
for date in datelist:
    print('Updating SWE predictions for ', date)
    #connect interactive script to Wasatch Snow module
    Snow = NSM_SCA.NSM_SCA(cwd, date)
    
    #Go get SNOTEL observations -- currently saving to csv, change to H5,
    #dd if self.data < 11-1-2022 and SWE = -9999, 
    Snow.Get_Monitoring_Data_Threaded()

    #Get the prediction extent
    bbox = Snow.getPredictionExtent()

    #Initialize/Download the granules
    Snow.initializeGranules(bbox, Snow.SCA_folder)

    #Process observations into Model prediction ready format,
    Snow.Data_Processing()

    #Agument with SCA
    Snow.augmentPredictionDFs()

    #Make predictions
    Snow.SWE_Predict()

    #Make CONUS netCDF file, compressed.
    Snow.netCDF_compressed(plot=False)

    #Make GeoDataframe and plot, self.Geo_df() makes the geo df
    # Snow.Geo_df()
    # Snow.plot_interactive_SWE_comp(pinlat = 39.1, pinlong = -120, web = True)

In [ ]:
def run(cwd, date):
    print('Updating SWE predictions for ', date)
    #connect interactive script to Wasatch Snow module
    Snow = NSM_SCA.NSM_SCA(cwd, date)

    #Go get SNOTEL observations -- currently saving to csv, change to H5,
    #dd if self.data < 11-1-2022 and SWE = -9999,
    Snow.Get_Monitoring_Data_Threaded()

    #Get the prediction extent
    bbox = Snow.getPredictionExtent()

    #Initialize/Download the granules
    Snow.initializeGranules(bbox, Snow.SCA_folder)

    #Process observations into Model prediction ready format,
    Snow.Data_Processing()

    #Agument with SCA
    Snow.augmentPredictionDFs()

    #Make predictions
    Snow.SWE_Predict()

    #Make CONUS netCDF file, compressed.
    Snow.netCDF_compressed(plot=False)

In [ ]:
for date in datelist[:4]:
    %time run(cwd, date)